In [1]:
import pandas as pd
from apyori import apriori
from nltk.stem import WordNetLemmatizer, PorterStemmer
from datetime import datetime

def PrintRuleAssociation():
    bow = []
    
    porter = PorterStemmer()
    lemmatizer = WordNetLemmatizer()
    
    bow = GetTopicTerms(lda, word) 
    
    if len(bow) == 0:
        print('No topics found')
        return
    
    rules = apriori(bow, min_support = 0.2, min_confidence = 0.9)
    results = list(rules)
    
    df = pd.DataFrame(columns=('Left Hand Side','Right Hand Side','Support','Confidence','Lift'))
    
    Support =[]
    Confidence = []
    Lift = []
    Items = []
    Antecedent = []
    Consequent=[]
      
    for RelationRecord in results:
        for ordered_stat in RelationRecord.ordered_statistics:
            consequences = list(ordered_stat.items_base)
            antecedent = list(ordered_stat.items_add)
            if ((covid in consequences or covid in antecedent) or (sarscov in consequences or sarscov in antecedent) or (infect in consequences or infect in antecedent)) \
            and (word in consequences or word in antecedent) \
            and (severe in consequences or severe in antecedent):
                Support.append(RelationRecord.support)
                Antecedent.append(ordered_stat.items_base)
                Consequent.append(ordered_stat.items_add)
                Confidence.append(ordered_stat.confidence)
                Lift.append(ordered_stat.lift)

                                          
    df['Left Hand Side'] = list(map(set, Antecedent))
    df['Right Hand Side'] = list(map(set, Consequent))
    df['Support'] = Support
    df['Confidence'] = Confidence
    df['Lift'] = Lift
    df.sort_values(by ='Lift', ascending = False, inplace = True)
    print(df)
    display(HTML(df.to_html()))

In [2]:
import gensim

def ngrams(words, minimumCount=5, threshold=100):
    bigram = gensim.models.Phrases(words, min_count=minimumCount, threshold=threshold) # higher threshold fewer phrases.
    trigram = gensim.models.Phrases(bigram[words], threshold=threshold)  

    bigram_mod = gensim.models.phrases.Phraser(bigram)
    trigram_mod = gensim.models.phrases.Phraser(trigram)
    return bigram_mod, trigram_mod


In [72]:
#needed to do this: pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib
#this may have been a waste of time

from __future__ import print_function
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import io

# If modifying these scopes, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/drive.metadata.readonly']

def GetDataFromGoogleDrive():
    """Shows basic usage of the Drive v3 API.
    Prints the names and ids of the first 10 files the user has access to.
    """
    creds = None
    # The file token.pickle stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('drive', 'v3', credentials=creds)

    #get metadata
    # Call the Drive v3 API
    results = service.files().list(
        pageSize=10, fields="nextPageToken, files(id, name)").execute()
    items = results.get('files', [])

    #print(type(items))
    #document = service.documents().get(documentId=DOCUMENT_ID).execute()

    #print('The title of the document is: {}'.format(document.get('title')))
    #print(items)

    file_id = '1dDxbn8_NcmihPQMeE7wzIuwRp0huPCpq'
    
    request = service.files().export_media(fileId=file_id,
                                           sheet = service.spreadsheets(),
                                           r = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                                range=SAMPLE_RANGE_NAME).execute()
    values = r.get('values', [])

SyntaxError: invalid syntax (<ipython-input-72-e2661e5f444c>, line 56)

In [73]:
t = GetDataFromGoogleDrive()


AttributeError: 'Resource' object has no attribute 'spreadsheets'

In [16]:
from collections import defaultdict
import csv
from pathlib import Path

#change this to take a date time range and check for the pmc files because they have a cleaner textbody
#we no longer need the abstract to filter out based on keywords
def GetData(startDate, endDate):
    data = defaultdict(list)   
    listofarticles = []
    try:
        with open('metadata.csv') as f_in:
            reader = csv.DictReader(f_in)
            for row in reader:
                if '-' not in row['publish_time']:
                    continue
                elif not (startDate <= datetime.strptime(row['publish_time'], '%Y-%m-%d') <= endDate):
                    continue
                if not row['pmc_json_files']:
                    continue   
            
                for json_path in row['pmc_json_files'].split(';'):
                    json_file = Path(json_path)
                    if json_file.is_file():
                        listofarticles.append(json_path)
                        data[row['cord_uid']] = True
                
    except ValueError:
        print("An error occurred: ", ValueError, " Please try again.")
    return listofarticles

In [17]:
import json
import re

def GetTextBodies(listOfpdfs):    
    text = []    
    for json_path in listOfpdfs:
        with open(json_path.replace(" ", "")) as f_json:
            full_text_dict = json.load(f_json)
            textBody = []
            for paragraph_dict in full_text_dict['body_text']:          
                paragraph_text = re.sub(r'[^a-zA-Z_\s]+', '', paragraph_dict['text'])             
                textBody.append(paragraph_text)
        text.append(paragraph_text)
    return text

In [18]:
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize  
from nltk.stem import WordNetLemmatizer, PorterStemmer

def RemoveStopWords(listOfDocs): 
    porter = PorterStemmer()
    lemmatizer = WordNetLemmatizer()
    stop_words = stopwords.words('english')
    
    resultDocs = []
    words = set(nltk.corpus.words.words())

    for doc in listOfDocs:
        result = []
        for word in doc.split(' '):
            lowerCasedWord = word.lower()
            lemmedWord = lemmatizer.lemmatize(lowerCasedWord)
                          
            if lemmedWord not in stop_words and lemmedWord not in "" and len(lemmedWord) > 3:
                stemmedWord = porter.stem(lemmedWord)
                if stemmedWord not in stop_words:
                    result.append(porter.stem(lemmedWord))
        resultDocs.append(result)
    return resultDocs

In [19]:
from gensim.corpora.dictionary import Dictionary

def ConvertDataToCorpus(cleaned_data):
    dictionary = Dictionary(cleaned_data)
    bigramMod, trigramMod = ngrams(cleaned_data)
    
    ngram =  [trigramMod[bigramMod[review]] for review in cleaned_data]
    
    id2word = gensim.corpora.Dictionary(ngram)
    id2word.compactify()
    
    corpus = [id2word.doc2bow(text) for text in ngram]
    
    return corpus, id2word, dictionary

In [20]:
#re-evaluate the parameters here, mess with the hyper parameter
def GetLDAModel(corpus, id2word,dictionary, numberOfTopics = 10, chunkSize=2000, passes=10):

    temp = dictionary[0]  

    return gensim.models.ldamulticore.LdaMulticore(corpus=corpus,
                                                   num_topics=numberOfTopics,
                                                   id2word=id2word,
                                                   chunksize=chunkSize,
                                                   workers=5, # Num. Processing Cores - 1
                                                   passes=passes,
                                                   eval_every = 1,
                                                   per_word_topics=True)

In [21]:
#change this to get only top topics
def GetTopicTerms(lda):
    bow = []

    for topic in lda.show_topic():
        print(topic)
        bow.append(list(map(lambda x: x[0], topic)))   
        
    return bow

In [ ]:
#add functionality to read from the google docs api



In [23]:
#split this up in a way that's easier to test, try splitting it up just a month at a time, find a way to visual topics
#per slice

import datetime
from pprint import pprint


print(datetime.datetime.now())

listOfpdfs= GetData(datetime.date(2020, 3, 1), datetime.date(2020, 3, 15))
    
if len(listOfpdfs) == 0:
    print("No PDFs found under this topic")
    
print('Amount of pdfs gathered: ', len(listOfpdfs))
data = GetTextBodies(listOfpdfs)
cleaned_data = RemoveStopWords(data)
corpus, id2word, dictionary = ConvertDataToCorpus(cleaned_data)
lda = GetLDAModel(corpus, id2word, dictionary, numberOfTopics = 40,chunkSize=2000)
lda.show_topics()
#PrintRuleAssociation(word, 3)
    
print(datetime.datetime.now())

2020-12-17 09:59:09.698087


FileNotFoundError: [Errno 2] No such file or directory: 'metadata.csv'

In [ ]:
lda.show_topics()